In [2]:
cd "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/minian"

C:\Users\Manip2\SCRIPTS\CodePythonAudrey\CodePythonAurelie\HayLabAnalysis\minian


In [3]:
# # Associate Ca2+ signal with sleep stages for each session & subsessions using crossregistration

#######################################################################################
                            # Define Experiment type #
#######################################################################################

#DrugExperiment=1 if CGP Experiment // DrugExperiment=0 if Baseline Experiment
DrugExperiment=0
#Sleep scoring from '_AB' '_AH' or initial ''
suffix='_AB' 
AnalysisID='TEST' 

#######################################################################################
                                # Load packages #
#######################################################################################

import os
import numpy as np
import quantities as pq
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import sys 
from datetime import datetime
import shutil
from ast import literal_eval
from scipy.signal import find_peaks
from scipy.stats import zscore
from itertools import groupby
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

minian_path = os.path.join(os.path.abspath('.'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))
sys.path.append(minian_path)


from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
#######################################################################################
                                # Define functions #
#######################################################################################

def Convert(string):
            li = list(string.split(", "))
            li2 = len(li)
            return li2

def find_session_folders(root_path):
    sessions = []
    sessions_path=[]
    # Iterate through items in the root_path
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        if os.path.isdir(item_path):
            # Check if the directory name contains "session"
            if "session" in item:
                sessions.append(item)
                sessions_path.append(item_path)
            else:
                # Check the subdirectories of the current directory
                for sub_item in os.listdir(item_path):
                    sub_item_path = os.path.join(item_path, sub_item)
                    if os.path.isdir(sub_item_path) and "session" in sub_item:
                        sessions.append(sub_item)
                        sessions_path.append(sub_item_path)
                        
    return sessions, sessions_path

#######################################################################################
                # Load sleep score and Ca2+ time series numpy arrays #
#######################################################################################

MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK','Purple' ,'ThreeColDotsOK'] if DrugExperiment else ['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'Purple', 'RedLinesOK','ThreeColDotsOK', 'ThreeBlueCrossesOK']

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")

destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/AB_Analysis/VigSt_{FolderNameSave}{suffix}{AnalysisID}" if DrugExperiment else f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/VigStates_{FolderNameSave}{suffix}{AnalysisID}"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/12_13_AssociateMinianSleepScore_FullAuto.py"
destination_file_path = f"{destination_folder}/12_13_AssociateMinianSleepScore_FullAuto.txt"
shutil.copy(source_script, destination_file_path)

for mice in MiceList:
    # Load sleep score and Ca2+ time series numpy arrays
    dpath0 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/" if DrugExperiment else "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/"
    dpath=dpath0 + mice
    print(f"####################################################################################")
    print(f"################################### {mice} ####################################")
    print(f"####################################################################################")
    print(f"Path to the folder : {dpath}")
    folder_base = Path(dpath)

    mfile = open(folder_base / f'mappingsAB_ALL.pkl', 'rb')
    mapping = pickle.load(mfile)

    sessions = []
    subsessions = []
    nb_minian_total=0
    dict_Calcium = {}
    dict_Spike = {}
    dict_Scoring = {}
    dict_Stamps = {}
    dict_TodropFile = {}
    dict_StampsMiniscope = {}
    dict_Path={}

    sessions, sessions_path = find_session_folders(folder_base)
    nb_sessions=len(sessions)

    for sess,session in enumerate(sessions):  
        session_path=Path(sessions_path[sess])
        folder_mini = session_path / f'V4_Miniscope'
        nb_subsessions = sum(1 for p in folder_mini.iterdir() if p.is_dir() and p.name.startswith("session"))
        ScoringFile = session_path/ f'OpenEphys/ScoredSleep{suffix}.npy'
        StampsFile = session_path/ f'SynchroFile.xlsx'
        StampsMiniscopeFile = folder_mini / f'timeStamps.csv'

        if nb_subsessions!=0:
            for x in range(1, nb_subsessions+1):            
                subsession= session + str(x)
                subsessions.append(subsession)    
                minian_ds = open_minian(folder_mini / subsession / f'minian')      # OR minianAB
                dict_Path[subsession] = session_path
                dict_Calcium[subsession] = minian_ds['C'] # calcium traces 
                dict_Spike[subsession] = minian_ds['S'] # estimated spikes
                dict_Scoring[subsession]  = np.load(ScoringFile)
                dict_Stamps[subsession]  = pd.read_excel(StampsFile)
                dict_StampsMiniscope[subsession]  = pd.read_csv(StampsMiniscopeFile)
                try:
                    TodropFile = folder_mini / subsession / f'minian/TodropFileAB.json'
                    with open(TodropFile, 'r') as f:
                        unit_to_drop = json.load(f)
                        dict_TodropFile[subsession]  = unit_to_drop
                except:
                    TodropFile = folder_mini / subsession / f'minian/TodropFile.json'
                    with open(TodropFile, 'r') as f:
                        unit_to_drop = json.load(f)
                        dict_TodropFile[subsession]  = unit_to_drop
                nb_minian_total+=1
        else:
            minian_ds = open_minian(folder_mini / f'minian')            # OR minianAB
            dict_Path[session] = session_path
            dict_Calcium[session] = minian_ds['C'] # calcium traces 
            dict_Spike[session] = minian_ds['S'] # estimated spikes
            dict_Scoring[session]  = np.load(ScoringFile) 
            dict_Stamps[session]  = pd.read_excel(StampsFile)
            dict_StampsMiniscope[session]  = pd.read_csv(StampsMiniscopeFile)
            try:
                TodropFile = folder_mini / f'minian/TodropFileAB.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[session]  = unit_to_drop
            except:
                TodropFile = folder_mini / f'minian/TodropFile.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[session]  = unit_to_drop
            nb_minian_total+=1  

    #######################################################################################
                            # Cross registration results #
    #######################################################################################
    
    B = mapping['session']    
    if mice == 'Purple' and DrugExperiment==0:
        index = B.columns
        B.columns = index.str.replace('part', 'session2')
    #######################################################################################
      # Distribute Ca2+ intensity & spikes to vigilance states for each sessions/subsessions #
    #######################################################################################
    
    data = {}
    counter=0
    VigilanceState_GlobalResults= pd.DataFrame(data, columns=['Mice','Session', 'Session_Time', 'Unique_Unit','UnitNumber','UnitValue', 'Drug', 'Substate','SubstateNumber','DurationSubstate', 'CalciumActivity', 'Avg_CalciumActivity', 'AUC_calcium','Avg_AUC_calcium', 'DeconvSpikeMeanActivity', 'Avg_DeconvSpikeActivity', 'SpikeActivityHz', 'Avg_SpikeActivityHz'])

    previousEndTime=0
    InitialStartTime=0

    CaCorrWakeMatrixBaseline=[]
    CaCorrNREMMatrixBaseline=[]
    CaCorrN2MatrixBaseline=[]
    CaCorrREMMatrixBaseline=[]

    SpCorrWakeMatrixBaseline=[]
    SpCorrNREMMatrixBaseline=[]
    SpCorrN2MatrixBaseline=[]
    SpCorrREMMatrixBaseline=[]

    RawCaTracesWake_Baseline=[]
    RawCaTracesNREM_Baseline=[]
    RawCaTracesN2_Baseline=[]
    RawCaTracesREM_Baseline=[]

    RawSpTracesWake_Baseline=[]
    RawSpTracesNREM_Baseline=[]
    RawSpTracesN2_Baseline=[]
    RawSpTracesREM_Baseline=[]

    CaCorrWakeMatrixCGP=[]
    CaCorrNREMMatrixCGP=[]
    CaCorrN2MatrixCGP=[]
    CaCorrREMMatrixCGP=[]

    SpCorrWakeMatrixCGP=[]
    SpCorrNREMMatrixCGP=[]
    SpCorrN2MatrixCGP=[]
    SpCorrREMMatrixCGP=[]

    RawCaTracesWake_CGP=[]
    RawCaTracesNREM_CGP=[]
    RawCaTracesN2_CGP=[]
    RawCaTracesREM_CGP=[]

    RawSpTracesWake_CGP=[]
    RawSpTracesNREM_CGP=[]
    RawSpTracesN2_CGP=[]
    RawSpTracesREM_CGP=[]

    Drugs=['Baseline', 'CGP'] if DrugExperiment else ['Baseline']

    filenameOut = folder_to_save / f'VigSt_CaCorr_{mice}.xlsx'
    excel_writerCa = pd.ExcelWriter(filenameOut)
    
    filenameOut = folder_to_save / f'VigSt_SpCorr_{mice}.xlsx'
    excel_writerSp = pd.ExcelWriter(filenameOut)

    filenameOut = folder_to_save / f'VigSt_RawCaTraces_{mice}.xlsx'
    excel_writerRawCa = pd.ExcelWriter(filenameOut)
    
    filenameOut = folder_to_save / f'VigSt_RawSpTraces_{mice}.xlsx'
    excel_writerRawSp = pd.ExcelWriter(filenameOut)

    for session in list(dict_Stamps.keys()):

        drug=os.path.basename(os.path.dirname(dict_Path[session])) if DrugExperiment else 'Baseline'

        # Start time & freq miniscope
        StartTime = list(dict_Stamps[session][0])[0]
        minian_freq=list(dict_Stamps[session][0])[2]
       
        C=dict_Calcium[session]
        Cupd = C.loc[:, :]
        rec_dur = Cupd.shape[1]
        S=dict_Spike[session] 
        Supd = S.loc[:, :] 

        # Adjust the StartTime if subsessions

        if InitialStartTime==0:
            InitialStartTime=StartTime    
            StartTimeMiniscope=0 # start time of miniscope rec of that subsesssions relative to the start of the mniscope recording
        else:
            if StartTime == InitialStartTime:
                StartTime = previousEndTime + 1/minian_freq #  +1 frame in seconds
                StartTimeMiniscope= StartTime-InitialStartTime
            else:  
                InitialStartTime=StartTime
                StartTimeMiniscope=0   

        # Deal with dropped frames (failure to acquire miniscope images)

        list_droppedframes = literal_eval(dict_Stamps[session][0][3])    

        numbdropfr= 0   
        upd_rec_dur=rec_dur
        droppedframes_inrec=[]
        for item in list_droppedframes: 
            if item < (int(StartTimeMiniscope*minian_freq) + upd_rec_dur) and item > int(StartTimeMiniscope*minian_freq):
                droppedframes_inrec.append(item-int(StartTimeMiniscope*minian_freq))
                upd_rec_dur+=1 #add the dropped frame to the recording length
                numbdropfr+=1                        

        EndTime = StartTime + (upd_rec_dur/minian_freq) # in seconds
        previousEndTime=EndTime 

        print(session, ': starts at', round(StartTime,1), 's & ends at', round(EndTime,1), 's (', round(upd_rec_dur/minian_freq,1), 's duration, ', numbdropfr, 'dropped frames, minian frequency =', minian_freq, 'Hz, drug = ', drug, ')...') 

        # Remove bad units from recordings

        AA = C['unit_id']
        copyAA = list(AA.copy())
        unit_to_drop=dict_TodropFile[session]    
        for u in unit_to_drop: # ugly way to do it, need to be improved to only use unit_to_drop
            copyAA.remove(u)
        unit_to_keep = copyAA
        Cupd = Cupd.loc[unit_to_keep,:]
        Carray = Cupd.values.T
        Supd = Supd.loc[unit_to_keep,:]
        Sarray = Supd.values.T
        nb_unit = Cupd.shape[0]
        units = range(nb_unit)
                        
        C_upd_unit_id = Cupd['unit_id'].values
        kept_uniq_unit_List=[]
        for unit in units:
            indexMapp = np.where(B[session] == C_upd_unit_id[unit])[0]
            kept_uniq_unit_List.append(str(indexMapp))

        sentence1= f"... kept values = {kept_uniq_unit_List}"
        print(sentence1)    

        if nb_unit==0:
            continue  # next iteration

        # Normalize traces
        """
        Carray=zscore(Carray, axis=0)
        min=np.min(Carray,axis=0) 
        Carray=Carray-min
        Sarray=zscore(Sarray, axis=0)
        min=np.min(Sarray,axis=0) 
        Sarray=Sarray-min
        """
        # Replace dropped frame in calcium and spike traces with the previous value

        for droppedframe in droppedframes_inrec: 
            row_to_repeat = Carray[droppedframe]  
            Carray = np.vstack((Carray[:droppedframe], row_to_repeat, Carray[droppedframe:]))
            row_to_repeat = Sarray[droppedframe]  
            Sarray = np.vstack((Sarray[:droppedframe], row_to_repeat, Sarray[droppedframe:]))

        # Upscale scoring to miniscope frequency

        scale_factor=minian_freq/0.2  #cause scoring was done in 5 seconds bin, ie 0.2 Hz
        SleepScoredTS=dict_Scoring[session]
        SleepScoredTS_upscaled = np.repeat(SleepScoredTS, scale_factor, axis=0)
        StartTime_frame=int(StartTime*minian_freq)
        SleepScoredTS_upscaled_ministart=SleepScoredTS_upscaled[StartTime_frame:StartTime_frame+upd_rec_dur]

        # Remove N2 stage
        #SleepScoredTS_upscaled_ministart[SleepScoredTS_upscaled_ministart == 0.5] = 0
            
        # Determine each substate identity and duration
        array=SleepScoredTS_upscaled_ministart
        substates_duration = [len(list(group)) for key, group in groupby(array)]
        substates_identity = [key for key, _ in groupby(array)]
        substates_end = np.array(substates_duration).cumsum()        
        substates_start =np.append([0],substates_end[:-1]) #substates_start =np.append([1],substates_end+1) create 1 second gap
        mapp = {1.5: 'Wake', 0.5: 'N2', 0: 'NREM',  1: 'REM'}
        substates_identity = [mapp[num] for num in substates_identity]
        substates = pd.DataFrame(list(zip(substates_identity, substates_duration, substates_start, substates_end)), columns=['Identity', 'Duration', 'Start','End'])

        for m in mapp:

            Bool = (SleepScoredTS_upscaled_ministart == m)
            Carray_VigSpe = Carray.copy()
            Carray_VigSpe = Carray_VigSpe[0:np.shape(SleepScoredTS_upscaled_ministart)[0],:] # if Calcium imaging longer than LFP rec
            Carray_VigSpe = Carray_VigSpe[Bool, :]

            RawCaTracesVigStateMatrixName=f'RawCaTraces{mapp[m]}_{drug}'
            RawCaTracesVigStateMatrix = locals()[RawCaTracesVigStateMatrixName]
            RawCaTraces=[]
            RawCaTraces=pd.DataFrame(Carray_VigSpe, columns=[f"{mice}{str(i).replace('[','').replace(']','')}" for i in kept_uniq_unit_List])
            unique_columns = RawCaTraces.columns[RawCaTraces.columns.to_series().duplicated()] # remove units that has an empty unique index '[]'
            RawCaTraces = RawCaTraces.drop(columns=unique_columns)
            RawCaTracesVigStateMatrix.append(RawCaTraces)

            Sarray_VigSpe = Sarray.copy()
            Sarray_VigSpe = Sarray_VigSpe[0:np.shape(SleepScoredTS_upscaled_ministart)[0],:] # if Calcium imaging longer than LFP rec
            Sarray_VigSpe = Sarray_VigSpe[Bool, :]         

            RawSpTracesVigStateMatrixName=f'RawSpTraces{mapp[m]}_{drug}'
            RawSpTracesVigStateMatrix = locals()[RawSpTracesVigStateMatrixName]
            RawSpTraces=[]
            RawSpTraces=pd.DataFrame(Sarray_VigSpe, columns=[f"{mice}{str(i).replace('[','').replace(']','')}" for i in kept_uniq_unit_List])
            unique_columns = RawSpTraces.columns[RawSpTraces.columns.to_series().duplicated()] # remove units that has an empty unique index '[]'
            RawSpTraces = RawSpTraces.drop(columns=unique_columns)
            RawSpTracesVigStateMatrix.append(RawSpTraces) 
            
            CaCorrVigStateMatrixName=f'CaCorr{mapp[m]}Matrix{drug}'
            CaCorrVigStateMatrix = locals()[CaCorrVigStateMatrixName]
            CaCorrMatrix=[]
            CaCorrMatrix = pd.DataFrame(columns=[f'{mice}{str(i)}' for i in range(len(B))], index=[i for i in range(len(B))])
            
            SpCorrVigStateMatrixName=f'SpCorr{mapp[m]}Matrix{drug}'
            SpCorrVigStateMatrix = locals()[SpCorrVigStateMatrixName]
            SpCorrMatrix=[]
            SpCorrMatrix = pd.DataFrame(columns=[f'{mice}{str(i)}' for i in range(len(B))], index=[i for i in range(len(B))])

            for unit in range(nb_unit): 

                Carray_unit =Carray_VigSpe[:,unit]
                Darray_unit =Sarray_VigSpe[:,unit] # on deconv spike not on spike rate                
                #peaks, _ = find_peaks(Darray_unit)
                #Sarray_unit=np.zeros(len(Darray_unit))
                #Sarray_unit[peaks]=1     

                otherunit_range = [x for x in range(nb_unit) if x != unit]

                for unit2 in range(nb_unit):

                    Carray_unit2 =Carray_VigSpe[:,unit2]
                    Darray_unit2 =Sarray_VigSpe[:,unit2]                    
                    #peaks2, _ = find_peaks(Darray_unit2)
                    #Sarray_unit2=np.zeros(len(Darray_unit2))
                    #Sarray_unit2[peaks2]=1

                    indexMapp = str(np.where(B[session] == C_upd_unit_id[unit])[0]).replace('[','').replace(']','')
                    indexMapp2 = np.where(B[session] == C_upd_unit_id[unit2])[0]

                    if any(indexMapp) and len(indexMapp2)>0:      
                                        
                        corr_matrix = np.corrcoef(Carray_unit, Carray_unit2)
                        CaCorrMatrix[f'{mice}{indexMapp}'][indexMapp2]={1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))
                        
                        corr_matrix = np.corrcoef(Darray_unit, Darray_unit2)
                        SpCorrMatrix[f'{mice}{indexMapp}'][indexMapp2]={1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))
                        
            CaCorrVigStateMatrix.append(CaCorrMatrix)
            SpCorrVigStateMatrix.append(SpCorrMatrix) 

        for unit in range(nb_unit): 

            Carray_unit =Carray[:,unit]
            Darray_unit =Sarray[:,unit]
            peaks, _ = find_peaks(Darray_unit)
            Sarray_unit=np.zeros(len(Darray_unit))
            Sarray_unit[peaks]=1     

            for index in range(len(substates)):
                
                ca_input_sub=Carray_unit[substates.Start[index]:substates.End[index]]
                ds_input_sub=Darray_unit[substates.Start[index]:substates.End[index]]
                sp_input_sub=Sarray_unit[substates.Start[index]:substates.End[index]]

                VigilanceState_GlobalResults.loc[counter, 'Mice'] = mice
                VigilanceState_GlobalResults.loc[counter, 'Session'] = session
                VigilanceState_GlobalResults.loc[counter, 'Session_Time'] = None 
                
                indexMapp = np.where(B[session] == C_upd_unit_id[unit])[0]
                VigilanceState_GlobalResults.loc[counter, 'Unique_Unit'] = indexMapp if len(indexMapp)>0 else None
                VigilanceState_GlobalResults.loc[counter, 'UnitNumber'] = unit
                VigilanceState_GlobalResults.loc[counter, 'UnitValue'] = C_upd_unit_id[unit]

                VigilanceState_GlobalResults.loc[counter, 'Drug'] =  os.path.basename(os.path.dirname(dict_Path[session])) if DrugExperiment else 'Baseline'

                VigilanceState_GlobalResults.loc[counter, 'Substate'] = substates.Identity[index]
                VigilanceState_GlobalResults.loc[counter, 'SubstateNumber'] = substates.index[index]
                VigilanceState_GlobalResults.loc[counter, 'DurationSubstate'] = (substates.Duration[index]/minian_freq)

                VigilanceState_GlobalResults.loc[counter, 'CalciumActivity'] = ca_input_sub.mean()
                VigilanceState_GlobalResults.loc[counter, 'Avg_CalciumActivity'] = Carray_unit.mean()

                VigilanceState_GlobalResults.loc[counter, 'AUC_calcium'] = np.trapz(ca_input_sub,np.arange(0,len(ca_input_sub),1))
                VigilanceState_GlobalResults.loc[counter, 'Avg_AUC_calcium'] = np.trapz(Carray_unit,np.arange(0,len(Carray_unit),1))

                VigilanceState_GlobalResults.loc[counter, 'DeconvSpikeMeanActivity'] = ds_input_sub.mean()
                VigilanceState_GlobalResults.loc[counter, 'Avg_DeconvSpikeActivity'] = Darray_unit.mean()

                VigilanceState_GlobalResults.loc[counter, 'SpikeActivityHz'] = sp_input_sub.sum()/(len(sp_input_sub)/minian_freq)
                VigilanceState_GlobalResults.loc[counter, 'Avg_SpikeActivityHz'] = Sarray_unit.sum()/(len(Sarray_unit)/minian_freq)

                otherunit_range = [x for x in range(nb_unit) if x != unit]

                CaCorrCoeff=[]
                SpCorrCoeff=[]
                Z_CaCorrCoeff=[]
                Z_SpCorrCoeff=[]

                for unit2 in otherunit_range:

                    Carray_unit2 =Carray[:,unit2]
                    Darray_unit2 =Sarray[:,unit2]                    
                    #peaks2, _ = find_peaks(Sarray_unit2)
                    #Sarray_unit2=np.zeros(len(Sarray_unit2))
                    #Sarray_unit2[peaks2]=1
                    ca_input_sub2=Carray_unit2[substates.Start[index]:substates.End[index]]
                    ds_input_sub2=Darray_unit2[substates.Start[index]:substates.End[index]]

                    corr_matrix = np.corrcoef(ca_input_sub, ca_input_sub2)
                    CaCorrCoeff_unit = np.round(corr_matrix[0, 1],5)
                    CaCorrCoeff_unit = {1: 0.99999, -1: -0.99999}.get(CaCorrCoeff_unit, CaCorrCoeff_unit)
                    CaCorrCoeff.append(CaCorrCoeff_unit)
                    Z_CaCorrCoeff_unit=np.arctanh(CaCorrCoeff_unit)
                    Z_CaCorrCoeff.append(Z_CaCorrCoeff_unit)

                    corr_matrix = np.corrcoef(ds_input_sub, ds_input_sub2)
                    SpCorrCoeff_unit =  np.round(corr_matrix[0, 1],5)
                    SpCorrCoeff_unit = {1: 0.99999, -1: -0.99999}.get(SpCorrCoeff_unit, SpCorrCoeff_unit)
                    SpCorrCoeff.append(SpCorrCoeff_unit)
                    Z_SpCorrCoeff_unit=np.arctanh(SpCorrCoeff_unit)
                    Z_SpCorrCoeff.append(Z_SpCorrCoeff_unit)
                
                VigilanceState_GlobalResults.loc[counter, 'CaCorrCoeff'] = np.nanmean(CaCorrCoeff)
                VigilanceState_GlobalResults.loc[counter, 'SpCorrCoeff'] = np.nanmean(SpCorrCoeff)
                
                VigilanceState_GlobalResults.loc[counter, 'Rsquared_CaCorrCoeff'] = np.nanmean([x ** 2 for x in CaCorrCoeff])
                VigilanceState_GlobalResults.loc[counter, 'Rsquared_SpCorrCoeff'] = np.nanmean([x ** 2 for x in SpCorrCoeff])

                VigilanceState_GlobalResults.loc[counter, 'Z_CaCorrCoeff'] = np.nanmean(Z_CaCorrCoeff)
                VigilanceState_GlobalResults.loc[counter, 'Z_SpCorrCoeff'] = np.nanmean(Z_SpCorrCoeff)
                
                VigilanceState_GlobalResults.loc[counter, 'Z_Rsquared_CaCorrCoeff'] = np.nanmean([x ** 2 for x in Z_CaCorrCoeff])
                VigilanceState_GlobalResults.loc[counter, 'Z_Rsquared_SpCorrCoeff'] = np.nanmean([x ** 2 for x in Z_SpCorrCoeff])   

                Carray_Population =np.mean(Carray[:,otherunit_range], axis=1)
                ca_input_sub2=Carray_Population[substates.Start[index]:substates.End[index]]
                corr_matrix = np.corrcoef(ca_input_sub, ca_input_sub2)
                VigilanceState_GlobalResults.loc[counter, 'CaPopCoupling'] = np.round(corr_matrix[0, 1],5)
                corCorrected = {1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))
                VigilanceState_GlobalResults.loc[counter, 'Z_CaPopCoupling'] = np.arctanh(corCorrected)

                Sarray_Population =np.mean(Sarray[:,otherunit_range], axis=1)
                ds_input_sub2=Sarray_Population[substates.Start[index]:substates.End[index]]
                corr_matrix = np.corrcoef(ds_input_sub, ds_input_sub2)
                VigilanceState_GlobalResults.loc[counter, 'SpPopCoupling'] = np.round(corr_matrix[0, 1],5)
                corCorrected = {1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))
                VigilanceState_GlobalResults.loc[counter, 'Z_SpPopCoupling'] = np.arctanh(corCorrected)

                counter+=1

    for Drug in Drugs:
        for m in mapp:
            CaCorrVigStateMatrixName=f'CaCorr{mapp[m]}Matrix{Drug}'
            CaCorrVigStateMatrix = locals()[CaCorrVigStateMatrixName]
            if len(CaCorrVigStateMatrix)>0: # cause sometimes no Baseline conditions in CGP experiments
                combined_df = pd.concat(CaCorrVigStateMatrix, ignore_index=False)
                IterationNb=combined_df.groupby(combined_df.index).count()
                combined_df = combined_df.groupby(combined_df.index).sum() #mean
                combined_df.index= [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerCa, sheet_name=f'{Drug}_{mapp[m]}', index=True, header=True) 
                
                combined_df = pd.concat(CaCorrVigStateMatrix, ignore_index=False)
                combined_df = combined_df.applymap(lambda x: np.arctanh(x) if not pd.isna(x) else np.nan)
                combined_df = combined_df.groupby(combined_df.index).sum() #mean
                combined_df.index = [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerCa, sheet_name=f'Z_{Drug}_{mapp[m]}', index=True, header=True)   

                IterationNb.to_excel(excel_writerCa, sheet_name=f'{Drug}_{mapp[m]}_IterationNb', index=True, header=True) 

                SpCorrVigStateMatrixName=f'SpCorr{mapp[m]}Matrix{Drug}'
                SpCorrVigStateMatrix = locals()[SpCorrVigStateMatrixName]
                combined_df = pd.concat(SpCorrVigStateMatrix, ignore_index=False)
                IterationNb=combined_df.groupby(combined_df.index).count()
                combined_df = combined_df.groupby(combined_df.index).sum()
                combined_df.index = [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerSp, sheet_name=f'{Drug}_{mapp[m]}', index=True, header=True) 
                
                combined_df = pd.concat(SpCorrVigStateMatrix, ignore_index=False)
                combined_df = combined_df.applymap(lambda x: np.arctanh(x) if not pd.isna(x) else np.nan)
                combined_df = combined_df.groupby(combined_df.index).sum()
                combined_df.index = [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerSp, sheet_name=f'Z_{Drug}_{mapp[m]}', index=True, header=True) 
                
                IterationNb.to_excel(excel_writerSp, sheet_name=f'{Drug}_{mapp[m]}_IterationNb', index=True, header=True) 
            
            RawCaTracesVigStateMatrixName=f'RawCaTraces{mapp[m]}_{Drug}'
            RawCaTracesVigStateMatrix= locals()[RawCaTracesVigStateMatrixName]
            if len(RawCaTracesVigStateMatrix)>0: # cause sometimes no Baseline conditions in CGP experiments
                combined_df = pd.concat(RawCaTracesVigStateMatrix, ignore_index=False)
                combined_df.index = [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerRawCa, sheet_name=f'{Drug}_{mapp[m]}', index=False, header=True)   

                RawSpTracesVigStateMatrixName=f'RawSpTraces{mapp[m]}_{Drug}'
                RawSpTracesVigStateMatrix= locals()[RawSpTracesVigStateMatrixName]
                combined_df = pd.concat(RawSpTracesVigStateMatrix, ignore_index=False)
                combined_df.index = [mice + str(idx) for idx in combined_df.index]
                combined_df = combined_df.dropna(axis=0, how='all')
                combined_df = combined_df.dropna(axis=1, how='all')
                combined_df.to_excel(excel_writerRawSp, sheet_name=f'{Drug}_{mapp[m]}', index=False, header=True)   

    excel_writerCa.close()
    excel_writerSp.close()
    excel_writerRawCa.close()
    excel_writerRawSp.close()

    filenameOut = folder_to_save / f'VigSt_Global_{mice}.xlsx'
    writer = pd.ExcelWriter(filenameOut)
    VigilanceState_GlobalResults.to_excel(writer)
    writer.close()



The folder used for minian procedures is : C:\Users\Manip2\SCRIPTS\CodePythonAudrey\CodePythonAurelie\HayLabAnalysis\minian\minian
####################################################################################
################################### BlackLinesOK ####################################
####################################################################################
Path to the folder : //10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/BlackLinesOK
session1 : starts at 3.8 s & ends at 895.5 s ( 891.7 s duration,  0 dropped frames, minian frequency = 20 Hz, drug =  Baseline )...
... kept values = ['[3]', '[5]', '[1]', '[0]', '[4]', '[2]']
session2 : starts at 3.5 s & ends at 939.7 s ( 936.1 s duration,  0 dropped frames, minian frequency = 20 Hz, drug =  Baseline )...
... kept values = ['[3]', '[9]', '[7]', '[5]', '[11]', '[15]', '[1]', '[8]', '[0]', '[4]', '[16]', '[17]', '[2]', '[10]', '[6]', '[12]', '[14]', '[1

KeyboardInterrupt: 

In [41]:
combined_df = pd.concat(CaCorrVigStateMatrix, ignore_index=False)
combined_df.groupby(combined_df.index).count()

,BlackLinesOK0,BlackLinesOK1,BlackLinesOK2,BlackLinesOK3,BlackLinesOK4,BlackLinesOK5,BlackLinesOK6,BlackLinesOK7,BlackLinesOK8,BlackLinesOK9,...,BlackLinesOK16,BlackLinesOK17,BlackLinesOK18,BlackLinesOK19,BlackLinesOK20,BlackLinesOK21,BlackLinesOK22,BlackLinesOK23,BlackLinesOK24,BlackLinesOK25
0,3,3,2,3,3,3,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
1,3,3,2,3,3,3,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
2,2,2,2,2,2,2,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
3,3,3,2,3,3,3,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
4,3,3,2,3,3,3,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
5,3,3,2,3,3,3,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
6,2,2,1,2,2,2,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
7,2,2,1,2,2,2,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
8,2,2,1,2,2,2,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0
9,2,2,1,2,2,2,2,2,2,2,...,1,1,1,1,0,1,0,1,0,0


In [42]:
combined_df = pd.concat(CaCorrVigStateMatrix, ignore_index=False)
combined_df = combined_df.groupby(combined_df.index).mean()
combined_df

,BlackLinesOK0,BlackLinesOK1,BlackLinesOK2,BlackLinesOK3,BlackLinesOK4,BlackLinesOK5,BlackLinesOK6,BlackLinesOK7,BlackLinesOK8,BlackLinesOK9,...,BlackLinesOK16,BlackLinesOK17,BlackLinesOK18,BlackLinesOK19,BlackLinesOK20,BlackLinesOK21,BlackLinesOK22,BlackLinesOK23,BlackLinesOK24,BlackLinesOK25
0,0.99999,-0.045023,0.738405,0.406903,0.524517,-0.01086,0.45626,0.11214,-0.13398,-0.039075,...,-0.04834,-0.0251,0.03238,0.04318,NaN,-0.04445,NaN,0.17455,NaN,NaN
1,-0.045023,0.99999,-0.0366,-0.03764,-0.026573,0.005157,-0.029295,-0.02887,0.04784,-0.013175,...,0.01246,-0.00758,-0.02369,-0.0337,NaN,-0.02317,NaN,-0.01635,NaN,NaN
2,0.738405,-0.0366,0.99999,0.474055,0.40906,-0.03689,0.19181,0.00449,-0.07516,-0.02414,...,-0.03559,-0.01863,-0.00085,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.406903,-0.03764,0.474055,0.99999,0.215003,0.03964,0.091515,0.34782,-0.138045,-0.042335,...,0.13966,-0.03227,0.47465,0.08596,NaN,-0.05599,NaN,-0.02208,NaN,NaN
4,0.524517,-0.026573,0.40906,0.215003,0.99999,0.004757,0.842225,0.19991,-0.124065,-0.03028,...,-0.02163,-0.02464,0.14363,-0.02776,NaN,-0.02305,NaN,0.02329,NaN,NaN
5,-0.01086,0.005157,-0.03689,0.03964,0.004757,0.99999,-0.01909,0.37861,-0.074635,0.27557,...,-0.01638,0.01612,-0.01629,-0.03445,NaN,0.02603,NaN,0.07821,NaN,NaN
6,0.45626,-0.029295,0.19181,0.091515,0.842225,-0.01909,0.99999,0.296465,-0.11089,-0.030935,...,0.01363,-0.02216,0.30128,0.21238,NaN,-0.05251,NaN,0.01571,NaN,NaN
7,0.11214,-0.02887,0.00449,0.34782,0.19991,0.37861,0.296465,0.99999,-0.11835,-0.0328,...,-0.02427,-0.01344,0.74611,0.35565,NaN,-0.04226,NaN,0.044,NaN,NaN
8,-0.13398,0.04784,-0.07516,-0.138045,-0.124065,-0.074635,-0.11089,-0.11835,0.99999,0.001905,...,0.04164,-0.03021,-0.06366,-0.08326,NaN,0.17827,NaN,-0.09398,NaN,NaN
9,-0.039075,-0.013175,-0.02414,-0.042335,-0.03028,0.27557,-0.030935,-0.0328,0.001905,0.99999,...,-0.01814,-0.00953,-0.0298,-0.0285,NaN,-0.00132,NaN,-0.01729,NaN,NaN


In [37]:
(-0.04296 + -0.02608 + -0.06603)/3

-0.04502333333333333

In [33]:
CaCorrVigStateMatrix[0]

,BlackLinesOK0,BlackLinesOK1,BlackLinesOK2,BlackLinesOK3,BlackLinesOK4,BlackLinesOK5,BlackLinesOK6,BlackLinesOK7,BlackLinesOK8,BlackLinesOK9,...,BlackLinesOK16,BlackLinesOK17,BlackLinesOK18,BlackLinesOK19,BlackLinesOK20,BlackLinesOK21,BlackLinesOK22,BlackLinesOK23,BlackLinesOK24,BlackLinesOK25
0,0.99999,-0.06603,0.67202,0.71757,0.36042,-0.06037,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.06603,0.99999,-0.054,-0.0391,-0.03282,0.05419,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.67202,-0.054,0.99999,0.61384,0.50913,-0.03523,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.71757,-0.0391,0.61384,0.99999,0.34425,-0.0027,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.36042,-0.03282,0.50913,0.34425,0.99999,-0.02018,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.06037,0.05419,-0.03523,-0.0027,-0.02018,0.99999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
combined_df = pd.concat(CaCorrVigStateMatrix, ignore_index=False)


KeyError: 0

In [22]:
combined_df = combined_df.set_index(['NbSession', 'ID'])
combined_df

BlackLinesOK0 BlackLinesOK1 BlackLinesOK2  \
NbSession ID                                                         
3         BlackLinesOK0        2.99997      -0.13507       1.47681   
          BlackLinesOK1       -0.13507       2.99997       -0.0732   
          BlackLinesOK2        1.47681       -0.0732       1.99998   
          BlackLinesOK3        1.22071      -0.11292       0.94811   
          BlackLinesOK4        1.57355      -0.07972       0.81812   
          BlackLinesOK5       -0.03258       0.01547      -0.07378   
          BlackLinesOK6        0.91252      -0.05859       0.19181   
          BlackLinesOK7        0.22428      -0.05774       0.00449   
          BlackLinesOK8       -0.26796       0.09568      -0.07516   
          BlackLinesOK9       -0.07815      -0.02635      -0.02414   
          BlackLinesOK10       0.61484      -0.05919       0.15944   
          BlackLinesOK11       0.74901      -0.03661       0.24865   
          BlackLinesOK12       0.42497      -0.05971       0.04325   
          BlackLinesOK13             0             0             0   
          BlackLinesOK14      -0.04372      -0.01305      -0.03219   
          BlackLinesOK15       0.72712      -0.01336       0.62182   
          BlackLinesOK16      -0.04834       0.01246      -0.03559   
          BlackLinesOK17       -0.0251      -0.00758      -0.01863   
          BlackLinesOK18       0.03238      -0.02369      -0.00085   
          BlackLinesOK19       0.04318       -0.0337             0   
          BlackLinesOK20             0             0             0   
          BlackLinesOK21      -0.04445      -0.02317             0   
          BlackLinesOK22             0             0             0   
          BlackLinesOK23       0.17455      -0.01635             0   
          BlackLinesOK24             0             0             0   
          BlackLinesOK25             0             0             0   

                         BlackLinesOK3 BlackLinesOK4 BlackLinesOK5  \
NbSession ID                                                         
3         BlackLinesOK0        1.22071       1.57355      -0.03258   
          BlackLinesOK1       -0.11292      -0.07972       0.01547   
          BlackLinesOK2        0.94811       0.81812      -0.07378   
          BlackLinesOK3        2.99997       0.64501       0.11892   
          BlackLinesOK4        0.64501       2.99997       0.01427   
          BlackLinesOK5        0.11892       0.01427       2.99997   
          BlackLinesOK6        0.18303       1.68445      -0.03818   
          BlackLinesOK7        0.69564       0.39982       0.75722   
          BlackLinesOK8       -0.27609      -0.24813      -0.14927   
          BlackLinesOK9       -0.08467      -0.06056       0.55114   
          BlackLinesOK10       0.38046       1.37459       0.47788   
          BlackLinesOK11       0.08797       1.71254      -0.04067   
          BlackLinesOK12       0.46405       0.44279       0.09088   
          BlackLinesOK13             0             0             0   
          BlackLinesOK14       0.37738      -0.04198       -0.0259   
          BlackLinesOK15       0.15174       0.31567      -0.02751   
          BlackLinesOK16       0.13966      -0.02163      -0.01638   
          BlackLinesOK17      -0.03227      -0.02464       0.01612   
          BlackLinesOK18       0.47465       0.14363      -0.01629   
          BlackLinesOK19       0.08596      -0.02776      -0.03445   
          BlackLinesOK20             0             0             0   
          BlackLinesOK21      -0.05599      -0.02305       0.02603   
          BlackLinesOK22             0             0             0   
          BlackLinesOK23      -0.02208       0.02329       0.07821   
          BlackLinesOK24             0             0             0   
          BlackLinesOK25             0             0             0   

                         BlackLinesOK6 BlackLinesOK7 BlackLinesOK8  \
NbSession ID                     

In [21]:
combined_df['ID']=[mice + str(idx) for idx in combined_df.index]
combined_df['NbSession']=3
combined_df

,BlackLinesOK0,BlackLinesOK1,BlackLinesOK2,BlackLinesOK3,BlackLinesOK4,BlackLinesOK5,BlackLinesOK6,BlackLinesOK7,BlackLinesOK8,BlackLinesOK9,...,BlackLinesOK18,BlackLinesOK19,BlackLinesOK20,BlackLinesOK21,BlackLinesOK22,BlackLinesOK23,BlackLinesOK24,BlackLinesOK25,NbSession,ID
0,2.99997,-0.13507,1.47681,1.22071,1.57355,-0.03258,0.91252,0.22428,-0.26796,-0.07815,...,0.03238,0.04318,0,-0.04445,0,0.17455,0,0,3,BlackLinesOK0
1,-0.13507,2.99997,-0.0732,-0.11292,-0.07972,0.01547,-0.05859,-0.05774,0.09568,-0.02635,...,-0.02369,-0.0337,0,-0.02317,0,-0.01635,0,0,3,BlackLinesOK1
2,1.47681,-0.0732,1.99998,0.94811,0.81812,-0.07378,0.19181,0.00449,-0.07516,-0.02414,...,-0.00085,0,0,0,0,0,0,0,3,BlackLinesOK2
3,1.22071,-0.11292,0.94811,2.99997,0.64501,0.11892,0.18303,0.69564,-0.27609,-0.08467,...,0.47465,0.08596,0,-0.05599,0,-0.02208,0,0,3,BlackLinesOK3
4,1.57355,-0.07972,0.81812,0.64501,2.99997,0.01427,1.68445,0.39982,-0.24813,-0.06056,...,0.14363,-0.02776,0,-0.02305,0,0.02329,0,0,3,BlackLinesOK4
5,-0.03258,0.01547,-0.07378,0.11892,0.01427,2.99997,-0.03818,0.75722,-0.14927,0.55114,...,-0.01629,-0.03445,0,0.02603,0,0.07821,0,0,3,BlackLinesOK5
6,0.91252,-0.05859,0.19181,0.18303,1.68445,-0.03818,1.99998,0.59293,-0.22178,-0.06187,...,0.30128,0.21238,0,-0.05251,0,0.01571,0,0,3,BlackLinesOK6
7,0.22428,-0.05774,0.00449,0.69564,0.39982,0.75722,0.59293,1.99998,-0.2367,-0.0656,...,0.74611,0.35565,0,-0.04226,0,0.044,0,0,3,BlackLinesOK7
8,-0.26796,0.09568,-0.07516,-0.27609,-0.24813,-0.14927,-0.22178,-0.2367,1.99998,0.00381,...,-0.06366,-0.08326,0,0.17827,0,-0.09398,0,0,3,BlackLinesOK8
9,-0.07815,-0.02635,-0.02414,-0.08467,-0.06056,0.55114,-0.06187,-0.0656,0.00381,1.99998,...,-0.0298,-0.0285,0,-0.00132,0,-0.01729,0,0,3,BlackLinesOK9
